In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from copy import deepcopy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from scipy import stats
from sklearn import preprocessing
from kmodes.kprototypes import KPrototypes
#from orion.contrib.envs import load_env
#load_env()
import boto3
#from orion.sources import S3Source
aws_bucket = 'kilimanjaro-prod-datalake'
s3 = boto3.client('s3')

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
trans = pd.read_csv('transaction_tidy.csv', index_col=0)
trans.head()

,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
0,2139607,S0016-8920840,2015-08-30,13:00:00,2015-08-30,220c0261086d7d2c254a9ba8d9339e6c15cb2792,United Kingdom,S61,90956,Clearance,0.0,1,58.16,15.9233,2015,8,Sunday,S0,58.16
1,2842275,S0016-9272310,2016-03-10,15:19:00,2016-03-10,375083e129410daa79593eb12317e1a3ead57f59,United Kingdom,ML2,87518,Clearance,0.0,1,63.94,10.9850,2016,3,Thursday,S0,63.94
2,2821100,S0016-9263541,2016-03-03,20:24:00,2016-03-03,8151f4728247ec548b4178b0d0685f70f45f3ec4,United Kingdom,S25,89813,Full Price,0.0,1,4.32,0.0000,2016,3,Thursday,S0,4.32
3,3128532,S0016-9408798,2016-06-08,23:10:00,2016-06-09,d42a941336443fc3070d9ef2a6c36982bffc1932,United Kingdom,EH6,100726,Full Price,0.0,1,33.61,14.0900,2016,6,Wednesday,S0,33.61
4,2874597,S0016-9288483,2016-03-22,13:10:00,2016-03-22,f25eaaea0e1c9e713308d7a9ae2624589742e7d0,United Kingdom,PR4,95335,Clearance,0.0,1,56.76,12.4700,2016,3,Tuesday,S0,56.76


In [4]:
cust_summary = pd.read_csv('cust_summary.csv', index_col=0)
cust_summary.head(1)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA,51.4728,-0.296367,UK


In [5]:
print(len(cust_summary))
print(cust_summary['customerkey'].nunique())

2204873
1570551


In [6]:
multi = cust_summary[cust_summary['orders']!= 1]

In [7]:
print(len(multi))
print(multi['customerkey'].nunique())

579624
415230


In [8]:
multi_custs = multi['customerkey'].values

In [9]:
trans_multi = deepcopy(trans[trans.customerkey.isin(multi_custs)])

In [10]:
print(trans_multi['customerkey'].nunique())

415230


In [11]:
trans_multi['orderdate'] = pd.to_datetime(trans_multi['orderdate'])

In [12]:
test_persons = deepcopy(trans_multi)

In [13]:
test_persons = test_persons[['customerkey','orderdate']]

In [14]:
test_persons = test_persons.sort_values(["customerkey", "orderdate"])

In [15]:
test_persons.dtypes

customerkey            object
orderdate      datetime64[ns]
dtype: object

In [16]:
test_persons = test_persons.drop_duplicates(
  subset = ['customerkey','orderdate'],
  keep = 'last').reset_index(drop = True)

In [17]:
test_persons = test_persons.sort_values(["customerkey", "orderdate"])

In [18]:
test_persons['previous_orderdate'] = test_persons.groupby(['customerkey'])['orderdate'].shift()

In [19]:
test_persons

,customerkey,orderdate,previous_orderdate
0,0000089aee56e015b7de440137bbd90baf2b6039,2018-07-20,NaT
1,0000089aee56e015b7de440137bbd90baf2b6039,2018-11-04,2018-07-20
2,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-02-05,NaT
3,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-06-06,2019-02-05
4,0000a804c45bb3ece53a155969e34813cfffdd6b,2016-08-01,NaT
...,...,...,...
1219584,fffeed94e03e4d07d52fdc5a98c03f1fcf981450,2017-11-28,2017-11-22
1219585,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2018-08-19,NaT
1219586,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2019-05-31,2018-08-19
1219587,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,2016-06-24,NaT


In [20]:
test_persons['days_bw_orders'] = test_persons['orderdate'] - test_persons['previous_orderdate']

In [22]:
test_persons

,customerkey,orderdate,previous_orderdate,days_bw_orders
0,0000089aee56e015b7de440137bbd90baf2b6039,2018-07-20,NaT,NaT
1,0000089aee56e015b7de440137bbd90baf2b6039,2018-11-04,2018-07-20,107 days
2,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-02-05,NaT,NaT
3,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-06-06,2019-02-05,121 days
4,0000a804c45bb3ece53a155969e34813cfffdd6b,2016-08-01,NaT,NaT
...,...,...,...,...
1219584,fffeed94e03e4d07d52fdc5a98c03f1fcf981450,2017-11-28,2017-11-22,6 days
1219585,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2018-08-19,NaT,NaT
1219586,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2019-05-31,2018-08-19,285 days
1219587,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,2016-06-24,NaT,NaT


In [23]:
test_persons['days_bw_orders']  = test_persons['days_bw_orders'] .apply(lambda x: x.days)

In [24]:
test_persons

,customerkey,orderdate,previous_orderdate,days_bw_orders
0,0000089aee56e015b7de440137bbd90baf2b6039,2018-07-20,NaT,NaN
1,0000089aee56e015b7de440137bbd90baf2b6039,2018-11-04,2018-07-20,107.0
2,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-02-05,NaT,NaN
3,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,2019-06-06,2019-02-05,121.0
4,0000a804c45bb3ece53a155969e34813cfffdd6b,2016-08-01,NaT,NaN
...,...,...,...,...
1219584,fffeed94e03e4d07d52fdc5a98c03f1fcf981450,2017-11-28,2017-11-22,6.0
1219585,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2018-08-19,NaT,NaN
1219586,ffff04fa0971b2f8f3caeb4d851987f28f50e28d,2019-05-31,2018-08-19,285.0
1219587,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,2016-06-24,NaT,NaN


In [34]:
order_lag = test_persons.groupby('customerkey').agg(days_bw_orders_mean=('days_bw_orders', 'mean'),days_bw_orders_median=('days_bw_orders', 'median'))
order_lag.head(2)

,days_bw_orders_mean,days_bw_orders_median
customerkey,,
0000089aee56e015b7de440137bbd90baf2b6039,107.0,107.0
0000167ab10ff470d5b1c2116b9aa33ddccc12bd,121.0,121.0


In [35]:
order_lag.to_csv('Order_lag.csv')